# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

* Ángel Torres del Álamo
* José Ángel Royo López

# 1. Introducción

En esta primera práctica vamos a realizar dos estudios utilizando los dataset `Diabetes` y `Wisconsin`. Para cada estudio se cargarán los datos, después se realizará un análisis exploratorio de datos mediante gráficas y estadísticos para obtener información que nos permita realizar un buen preprocesamiento de datos y aprender y evaluar varios modelos para estos conjuntos de datos. Para realizar esto se utilizará la librería `sklearn`, con la que nos iremos familiarizando para el desarrollo de las prácticas posteriores.

En primer lugar vamos a cargar librerias de python necesarias para realizar el estudio de forma comoda y correcta, con datos y gráficas que permitan sacar conclusiones. Tambíen utilizaremos el script realizado por `Juan Carlos Alfaro Jiménez` con otras funciones creadas por nosotros en el mismo archivo, que nos ayudará a hacer más legible el estudio.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.impute import KNNImputer
#from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline
from imblearn import FunctionSampler

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

# 2. Dataset Diabetes

Para este primer Dataset, vamos a tener un base de datos clínicos sobre resultados de la enfermedad de la diabetes y valores médicos relacionados con esta enfermedad.

Vamos a establecer una semilla para este estudio, para que los resultados que se muestran sean reproducibles y repetibles.

In [ ]:
seed = 27912

## 2.1. Acceso y almacenamiento de datos

Cargamos el conjunto de datos de diabetes, a través de la libreria de pandas.

In [ ]:
#filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
filepath = "../input/pimaindiansdiabetesdatabase/diabetes.csv"
target = 'Outcome'

data = utils.load_data(filepath, None, target)

Vamos a imprimir por pantalla 5 ejemplos de nuestra base de datos de forma aleatoria, para que nuestra muestra imprimida no esté `sesgada` y ver como se estructura la información.

In [ ]:
data.sample(5,random_state=seed)

Nuestra base de datos tiene 9 columnas o variables:

#### Variables predictoras
* `Pregnancies (Embarazos)`

* `Glucose (Glucosa)`

* `BloodPressure (Presión de sangre)`

* `SkinThickness (Espesor de la piel)`

* `Insulin (Insulina)`

* `BMI (IMC)`

* `DiabetesPedigreeFunction (Función de pedigrí de la diabetes)`

* `Age (Edad)`

#### Variable clase
* `Outcome (Resultado)`


Dividimos la base de datos en las variables predictoras, guardadas en el atributo `X`. Y la variable clase en el atributo `y`.

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Vamos a comprobar que la división de la base de datos se ha realizado de forma correcta y tiene todas las variables predictoras.

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.cat.categories

Vamos a separar el conjunto de datos en dos conjuntos diferentes: `Entrenamiento` y `test`.

Este proceso llamado `houldout`, lo utilizamos para no sobreajustar el modelo de entrenamiento y validar de una forma más honesta los resultados dados por el modelo.

* La muestra de entrenamiento será el 70% de la base de datos.
* La muestra de test será el 30% de la base de datos.

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Haciendo una muestra de la base de datos nueva de entrenamiento, vemos como es diferente a la muestra realizada anteriormente.

In [ ]:
X_train.sample(5, random_state=seed)

## 2.2. Análisis exploratorio

En este apartado, vamos a estudiar las diferentes variables que tenemos en nuestra base de datos. Así como los tipos de valores que tienen y los diferentes valores que pueden conseguir.  

Con el siguiente codigo, vamos a mostar la información de todas las variables de nuestra base de datos para ayudarnos en nuestro estudio.


In [ ]:
data.info(memory_usage=False)

La variable clase `Outcome` en este estudio es una variable numérica entera, discreta con dos posibles valores: 0 y 1. Este valor indica si dada la instancia de la base de datos, el resultado de su diagnostico es positivo en diabetes.

El resto de variables, las predictoras, tambien son numéricas de tipo entero. Excepto `BMI` y `DiabetePedigreeFunction`, que son de tipo flotante.

### 2.2.1 Visualización de los datos

Una vez conocidos los tipos de variables que tenemos vamos a estudiar los datos de las varaibles con más exactitud y sacar conclusiones razonables para la creación de un buen modelo.

Primero vamos a unir en una única base de datos, todo el conocimiento de entrenamiento que hemos obtenido anteriormente mediante el houldout de la base de datos completa.

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

Primero vamos a utilizar un estudio `univariado`. Al analizar las variables por separados podemos estudiar si alguna de estas variables contiene datos ruidosos o `outliers`. También podemos observar de esta manera la distribución de valores de una variable, si una variable es uniforme no nos aportará mucha información ya que tendrá siempre el mismo valor. Mientras que, si nuestra distribución es gaussiana nos ayudará a la hora de realizar el estudio.

A partir de un histograma podemos ver la densidad de ejemplos para distintos valores de una variable numérica. Tambíen nos permite conocer información de las variables por separado como hemos dicho anteriormente. La siguiente gráfica interactiva permite ver la distribución de cada variable por separado, pudiendo ver a simple vista valores ruidosos o la distribución de cada variable. En este caso, la gráfica lo utilizaremos para observar `outliers`.

In [ ]:
utils.plot_histogram(data_train)

Podemos observar en las gráficas como hay variables que poseen valores nulos, en este caso estos valores nulos estan representados por un valor `0`. Ya que, muchos atributos no tienen sentido porque no se puede tener este valor, pero hay algunos atributos que tener valores a 0 si tienen sentido. Lo estudiaremos a continuación.

Vamos a estudiar los atributos y si tienen *outliers* por orden que aparecen en la gráfica.

`Pregnancies`: Tiene algunso ejemplos en la base de datos que se aleja de la distribución de valores para este atributo. Esta variable al indicar los embarazos, tener valores tan altos como 14 embarazos, no tiene sentido. Aunque, como los casos son tan menores no se tendrán en cuenta.

`Glucose`: Este atributo poseé dos ejemplos con valores a 0. Lo cuál es un valor erroneo, ya que se necesita una medida. Este valor es un valor nulo para nuestra base de datos. Estos valores nulos los controlaremos en el preprocesamiento.

`BloodPressure`: Al igual que en Glucose también tiene valores a 0, a su vez tambíen hay muestras de datos que están alejados de la distribución normal de la variable.

`SkinThickness`: Posee muchos ejemplos en la base de datos con un valor a 0 o nulo. Esto supondrá un problema para el score que nos darán los modelos. En estos casos si el `20%` de los valores del atributo son nulos, eliminaremos la variable completa, ya que no introduce conocimiento adicional.

`Insulin`: Tienen muchos *outliers* que se alejan mucho de la distribución, también posee una gran cantidad de valores nulos. Es la variable con más valores a 0. Al igual que el anterior atributo, estudiaremos si la variable puede ser eliminada y en caso de que sea así lo haremos en el preprocesamiento.

`BMI`: Otro atributo que posee valores nulos que tendremos que manejar en el preprocesamiento.

`DiabetesPedigreeFunction`: Este atributo es similar a Insulin, tiene *outliers* muy alejados de la media de la distribución del atributo, aunque al ser tan pocos los casos no se eliminarán y es posible que estos outliers puedan ayudar a tener un modelo más preciso.

`Age`: Esta variable no supone muchos problemas con los *outliers*, teniendo muy pocos ejemplos de estos en la base de datos.

Como hemos comentado anteriormente, hay variables con gran cantidad de valores nulos. Vamos a dar el porcentaje que supone estos valores y si es `mayor que el 20%`, eliminaremos completamente todo el atributo. Estas variables son `SkinThickness` e `Insulin`. 

In [ ]:
valor_nulo = 0
data_train['SkinThickness'].value_counts(normalize = True)[valor_nulo] * 100

Para `SkinThickness`, el 29.24% de los valores del atributo son valores nulos, a 0. Entonces, esta variable la quitaremos de nuestra base de datos.

In [ ]:
data_train['Insulin'].value_counts(normalize = True)[valor_nulo] * 100

Para `Insulin`, el 48.6% de los valores son nulos, por tanto esta variable contiene una gran cantidad de estos valores y será eliminada.

In [ ]:
data_train['BloodPressure'].value_counts(normalize = True)[valor_nulo] * 100

Por último, para la variable `BloodPressure`, el 4.66% son valores nulos. Por tanto esta variable no será eliminada y realizaremos una imputación para dar un valor correcto a estos valores nulos en el preprocesamiento.

A continuación, vamos a estudiar la distribución de cada atributo. Podemos conocer cada distribución de una forma gráfica y ver que atributos se acercan a una distribución gaussiana o normal.

Con una función creada por nosotros, llamada `plot_distribution` y guardada en el script de `utils`, representaremos gráficamente la distribución de todas las variables para estudiarla de una forma más comoda. Ignorando los valores nulos y ruidosos que hemos visto en el anterior apartado.

In [ ]:
utils.plot_distribution(data_train)

Aunque en las gráficas salgan las variables Insulin y SkinThickness, no las estudiaremos ya que estos atributos los elimaneremos de nuestra base de datos como hemos comentado anteriormente. Iremos estudiando cada variable una por una, ya que a simple vista no podemos distinguir nada.

En las gráficas podemos observar distribuciones gaussianas o normales en forma de campana, estos atributos son: `BloodPressure`, `Glucose` y `BMI`.

Por último, para el análisis univariado, vamos a comprobar el número de casos diferentes para la variable clase. Comprobando si nuestra base de datos de entrenamiento esta `balanceada`.

In [ ]:
utils.plot_barplot(data_train)

In [ ]:
print("Valores:\n",data_train['Outcome'].value_counts(), "\n\nFrecuencia:\n",data_train['Outcome'].value_counts(normalize = True)*100)

El número de ejemplos en nuestra base de datos de entrenamiento esta `desbalanceado`. El número de resultados que son `0` es de **350 ejemplos**, mientras que de resultado `1` es de **187 ejemplos**. Esto supone que el `65.18%` de nuestra variable clase, es de valor `0`.

Es normal que en una base de datos medicos, tengamos más ejemplos de pacientes que no tengan una enfermedad dada. La mayoría de pacientes que se hacen revisiones no tendrán la enfermedad que estamos estudiando. Por tanto, es normal que este tipo de base de datos esten desbalanceadas.

Una vez estudiadas las variables de forma individual, para completar el estudio de las variables, tenemos que ver las relaciones entre estas variables. Este estudio llamado `multievaluado`, nos muestra información imporante sobre el discretizado de las variables y su potencia discriminatoria. Tambíen, sobre los valores ruidosos que se pueden encontrar en una zona con gran cantidad de otros valores clase.

Para este estudio se van a eliminar las variables `Insulin` y `SkinThickness` ya que no nos aportan información.

In [ ]:
utils.plot_pairplot(data_train[data_train.columns.difference(['Insulin','SkinThickness'])], target='Outcome')

Estás gráficas nos dan una representación de los valores de `Outcome` para cada par de variables. Podemos sacar un valor de corte y las variables necesarias para discretizar una variable clase y tener un modelo correcto.

Como se observa en las gráficas, los pares de variables no nos dan una discretización muy definida. Los valores de `Outcome` se superponen en los valores que vemos. Aunque, `Glucose` es la única variable que nos puede dar un potencial discriminatorio en valores altos de este atributo. Pero, contiene muchos valores ruidosos que dificultan un resultado preciso en el modelo final.

Entonces, como solo vemos un corte claro, la discretización se realizará mediante unicamente 2 intervalos en el caso de un modelo con discretización. Como tampoco se muestra un corte claro también se probará con un modelo sin discretizar.

## 2.3. Preprocesamiento de datos

El preprocesamiento de datos es la tarea más importante del proceso KDD. En este apartado realizaremos:

* `Limpieza de datos`
* `Integración de datos`
* `Transformación de datos`
* `Reducción de datos`

Para esta tarea se utilizara un `pipeline` para no cometer una `fuga de datos` y no introducir datos del conjunto de entrenamiento donde se aprenderá el modelo en el conjunto de prueba donde los datos de este conjunto son `datos crudos`. Una vez elaborado este apartado se tendrá que validad el modelo entrenado.

En primer lugar, quitaremos las columnas `SkinThickness` e `Insulin` mediante una función que le añadiremos al pipeline. Es necesario añadirla al pipeline, ya que al quitar columnas también es necesario quitarlas en la partición de test al igual que en la de entrenamiento.

In [ ]:
delete_colum = ['SkinThickness','Insulin']
del_col = FunctionTransformer(utils.drop_column,kw_args={'columns':delete_colum})

Estas columnas borradas ya no las tendremos en cuenta para los siguientes pasos que añadiremos al pipeline. Entonces, lo siguiente que realizamos es la `imputación` de los valores nulos. Estos valores que en nuestra base de datos están a `0`, serán reemplazados por valores con el algortimo de k-NN. Siendo k=5, para tener menos oportunidad de empate y tener valores más razonables para cada instancia de la base de datos al comprobar con 5 vecinos. Dado que para `Pregnancies` los valores a 0 si tienen sentido, tambíen quitaremos esta columna para la imputación.

In [ ]:
imputer_col = list(set(list(X)) - set(delete_colum) - set(['Pregnancies']))
print(imputer_col)

Estas son las columnas donde se hará la imputación y mediante *make_column_transformer* y el algoritmo imputador de k-NN, lo ejecutamos para las columnas que hemos indicado. Creando una función imputador que pasaremos al pipeline.

In [ ]:
imputer = make_column_transformer((KNNImputer(n_neighbors=5, weights="uniform",missing_values=0), imputer_col))

Utilizamos un discretizado en 2 particiones de forma uniforme. Según lo estudiado en el análisis es la mejor forma que tenemos para sacar buenos resultados.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="uniform")

## 2.4. Algoritmos de clasificación

Creamos un modelo 0-R para comparar con el resto de modelos.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

Creamos también un modelo de árbol de decisión que utilizaremos para comparar los resultados con nuestro pipeline creado. Como hemos dicho anteriormete la discretización puede causar que el modelo empeore ya que no hay un valor exacto por donde tendriamos una discretización visible, por tanto no crearemos un modelo de arbol discretizado fuera de nuestro pipeline.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Y por último, creamos el pipeline con las funciones vistas y creadas en el preprocesamiento: la función de quitado de columnas, el imputador para los valores nulos mediante k-NN, el discretizador y el modelo de árbol de decisión.

In [ ]:
discretize_pip = make_pipeline(del_col,imputer,discretizer,tree_model)

In [ ]:
pip = make_pipeline(del_col,imputer,tree_model)

## 2.5. Evaluación de modelos

Una vez creados todos los modelos, mediante la función `evaluate` del script `utils` 

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Acertando todos los valores de negativos de la base de datos tenemos la distribución de negativos en esta como valor de precisión.

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

Con un árbol de decisión mejoramos un poco la precisión obtenida, pudiendo esta vez predecir verdaderos positivos.

In [ ]:
utils.evaluate(pip,
               X_train, X_test,
               y_train, y_test)

Con la creación del pipeline que imputa valores nulos y elimina columnas que no aportan información, la precisión del modelo mejora, no muy significativamente. En este caso el algortimo utilizado es solamente el árbol de decisión.

In [ ]:
utils.evaluate(discretize_pip,
               X_train, X_test,
               y_train, y_test)

Como observamos, la diferencia de precisión entre el modelo discretizado y sin discretizar es muy pequeña con tan solo un valor de diferencia en la predicción. Por tanto, escogemos como modelo el pipeline discretizado creado para seguir con la evaluación.

In [ ]:
FP = 38  # False Positive
TP = 53  # True Positive

FN = 28  # False Negative
TN = 112 # True Negative

Tasa de falsos positivos: $$\frac{FP}{N} = \frac{FP}{FP + TN}$$

$$FP \rightarrow Número\ de\ Falsos\ Positivos$$
$$N \rightarrow Número\ de\ Negativos = FP + TN$$
$$TN \rightarrow Número\ de\ Verdaderos\ Negativos$$

In [ ]:
fp_rate= (FP)/(FP+TN)
print("Tasa de Falsos Positivos: ",fp_rate)

Tasa de falsos negativos: $$\frac{FN}{P} = \frac{FN}{FN + TP}$$

$$FN \rightarrow Número\ de\ Falsos\ Negativos$$
$$P \rightarrow Número\ de\ Positivos = FN + TP$$
$$TP \rightarrow Número\ de\ Verdaderos\ Positivos$$

In [ ]:
fn_rate = (FN)/(FN+TP)
print("Tasa de Falsos Negativos: ",fn_rate)

La tasa de falsos negativos es muy considerable, ya que esta al ser un `34.57%` 1 de cada 3 personas aproximadamente, siendo diabeticas, darán un resultado negativo.

Por otro lado, la tasa de falsos positivos es del `25.33%`, entonces 1 de cada 4 personas aproximadamente, darán positivo en diabetes sin ser una persona diabética. 

En este caso no tenemos ninguna decisión firme sobre la validez del modelo, ya que los ratios son altos y no podemos decantarnos por poner un ratio de FP o FN por encima del otro. Ya que, tener un diabético y no medicarlo es muy malo para ese paciente. Por otro lado, que una paciente no sea diabético y ponerle un tratamiento de una persona diabética como puede ser inyectarse insulina, puede acarrearle problemas de salud, aunque sea algo más beneficioso que el anterior.

Al realizar una evaluación del modelo y no tener claro si es un buen modelo o no mediante el estudio de la precisión o de la tasa de FP y FN, vamos a realizar una última evaluación que nos dará mayor claridad al problema dado. Vamos a ver estudiar si mediante el análisis de la `curva ROC` y el valor de `AUC`, que permite decidir a simple vista si un modelo es bueno. Nos permitirá analizar una relación de coste/beneficio 

In [ ]:
utils.evaluate_roc(discretize_pip,
    X_train, y_train,
    X_test,  y_test )

La curva ROC que nos produce nuestro modelo esta por encima de la línea diagonal (en la gráfica representada como una línea de puntos). Todo modelo que cumpla que esta condición, será un modelo mejor cuanto la curva más se acerque al punto que formaría con `(x=0,y=1)`,  Por tanto, a primera vista nuestro modelo es bueno al tener un ratio de Verdaderos Positivos más alto que de el ratio de Falsos Positivos. Cuanto más se máximice el valor de AUC (área bajo la curva), nuestro modelo será mejor. En este caso al tener un valor AUC de `0.7156` y visto que nuestra precisión del modelo no es muy alta y ningun modelo de los evaluados se acerca a una estimación mas exacta, este valor será de lo más alto que podemos conseguir. 

Por tanto, nuestro modelo entrenado será un buen modelo predictivo para el diagnostico de la diabetes.

# 3. Dataset Wisconsin

Antes de empezar con el estudio de este dataset, al igual que en el dataset anterior, vamos a establecer una semilla para que los resultados obtenidos sean reproducibles.

In [ ]:
seed = 27913

## 3.1. Acceso y almacenamiento de datos

Primero vamos a cargar el conjunto de datos con el que trabajaremos en esta parte de la práctica. Se trata del conjunto de datos `wisconsin`.

Los datos se encuentran almacenados en un fichero `.csv`, en el cada cada fila del fichero representa a una instancia, y en cada fila los datos están separados por coma, representando cada uno de estos datos separados por coma a una variable de nuestro problema.

Para cargar los datos en la libreta utilizaremos la función que se nos ha proporcionado. A esta función hay que indicarle cual es la variable, es decir, cual es la variable que queremos predecir. Para ello accedemos a [wisconsin](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data) donde podemos descargar el conjunto de datos `wisconsin` con el que vamos a trabajar y podemos ver que la variable `diagnosis` (diagnóstico) es la variable clase.

In [ ]:
filepath = "../input/wisconsin/wisconsin.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Vamos a mostrar algunas instancias para comprobar que se han cargado los datos correctamente. Mostraremos algunas al azar para evitar obtener una muestra sesgada que no represente al conjunto de datos.

In [ ]:
data.sample(5, random_state=seed)

Una vez tenemos los datos cargados correctamente vamos a dividir la base de datos en la variable clase `diagnosis` (`y`) y las variables predictoras (`X`).

In [ ]:
(X, y) = utils.divide_dataset(data, target=target)

Vamos a comprobar de la misma forma que antes que se han separado las variables correctamente.

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Ahora vamos a dividir los datos en datos de entrenamiento y datos de prueba para evitar usar un conjunto de los datos (el conjunto de test) en el entrenamiento de los modelos. Tampoco lo usaremos en ninguna de las etapas anteriores al entrenamiento. Esto lo realizamos para evitar un sobreajuste a los datos que tenemos, así el modelo aprenderá de los datos de entrenamiento, pero al no conocer los datos de prueba no puede ajustarse a ellos y generalizará mejor con datos nuevos, y estos los podremos usar para evaluar el modelo.

Estos datos de prueba serán usados exclusivamente para evaluar los modelos obtenidos.

In [ ]:
train_size = 0.7 # 70 % entrenamiento y 30 % de los datos test

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Con esto ya tenemos los datos almacenados correctamente y podemos empezar con la siguiente fase del proceso.

## 3.2. Análisis exploratorio

Ahora vamos a realizar un análisis exploratorio para observar como se comporta el conjunto de datos que tenemos, observando sus variables y la relación que tengan entre ellas y con la variable clase. Para ello analizaremos nuestro problema con el objetivo de obtener información relevante que nos sirva de utilidad para realizar un buen preprocesamiento de datos.

Indicar que este análisis exploratorio se realizará solo sobre los datos de entrenamiento, para no ajustarnos a los datos de prueba (evitarndo una ***fuga de datos***).

Primero vamos a empezar viendo el número de casos y de variables que tenemos en nuestros datos de entrenamiento.

In [ ]:
print(X_train.shape)
print(y_train.shape)

Podemos ver que tenemos 398 casos, y 32 variables en nuestros datos de entrenamiento, teniendo la variable clase y 31 variables predictoras.

Ahora vamos a ver de forma general como son las variables predictoras, es decir, si son variables discretas o continuas.

In [ ]:
X_train.info(memory_usage=False)

Podemos ver que todas las variables predictoras son numéricas (`float64`), esto quiere decir que son variables continuas ya que se representan mediante valores numéricos. También podemos ver que ninguna de las variables predictoras, excepto la última, tiene algún valor null, por lo que todos sus valores son conocidos.

La última variable predictora, llamada `Unnamed: 32`, se trata de una variable corrupta que tiene todos los valores ***perdidos***, es decir, todos sus valores valen `null`. Esto se debe a la última coma que aparece en la cabecera del fichero `csv` del que hemos cargado los datos. Esa coma provoca que se espere una variable después de la coma, pero como no hay ninguna la renombra a unnamed, y como no se le asigna ningún valor a esa variable, todos sus valores son perdidos (`null`).

Dado que esto es un error del fichero del que hemos obtenido los datos, y esta variable no aporta nada, la vamos a eliminar de los datos de prueba y de entrenamiento antes de seguir con el análisis exploratorio.

In [ ]:
X_train = X_train.drop("Unnamed: 32", axis=1)
X_test = X_test.drop("Unnamed: 32", axis=1)

X_train.info(memory_usage=False)

Ahora que hemos eliminado esta variable corrupta podemos ver que tenemos 30 variables predictoras. Estas variables predictoras aportan información acerca de las células, midiendo su:
* radius (radio)
* texture (textura)
* perimeter (perímetro)
* area (área)
* smoothness (suavidad)
* compactness (compacidad)
* concavity (concavidad)
* concave points (puntos cóncavos)
* symmetry (simetría)
* fractal dimmension (dimensión fractal)


Ahora vamos a ver la información de la variable clase.

In [ ]:
y_train

Podemos ver que la variable clase es discreta, estando formada por dos estados distintos que son ***B*** y ***M***. A grandes rasgos hemos podido ver que tenemos 30 variables predictoras para predecir si la variable objetivo (`diagnosis`) es B `benigno` o M `maligno`.

### Análisis univariado

Vamos a empezar viendo como de repartidas están las categorías de la variable clase realizando un análisis univariado.

Para ello primero vamos a generar una variable `data_train` que contenga solo los datos de entrenamiento, uniendo las variables predictoras y la objetivo para evitar obtener información en las gráficas de los datos de prueba.

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

Para empezar con el análisis univariado vamos a generar un gráfico para ver como de repartidas están las variables categóricas que tenemos, que en este caso la única variable categórica que tenemos es la variable clase.

In [ ]:
utils.plot_barplot(data_train)

Podemos ver que la etiqueta B es casi el doble de frecuente que la etiqueta M, por lo que tenemos una muestra desbalanceada, ya que no hay la misma proporción de las dos etiquetas.

Si multiplicamos el tanto por ciento que nos muestra la gráfica por la cantidad total de casos de prueba que tenemos (398), obtenemos la cantidad exacta de casos de cada etiqueta.

$$B = 0.6281407 * 398 = 250$$

$$M = 0.3718593 * 398 = 148$$

Podemos comprobar si esto es cierto utilizando el método `describe`.

In [ ]:
data_train.describe(include="category")

Acabamos de comprobar que la información obtenida a partir de los porcentajes de la gráfica es la correcta, teniendo efectivamente 250 casos de la categoría B, y el resto ($398-250 = 148$) de la categoría M.

Para seguir con este análisis univariado vamos a generar unos histogramas e intentar sacar la máxima información posible de las variables predictoras (recordemos que todas las variables predictoras de nuestro problema son numéricas).

In [ ]:
#utils.plot_distribution(data_train)

Con estos gráficos se puede ver de una forma sencilla las distribuciones que siguen todas las variables numéricas que tenemos.

Si vamos observando todas las variables podemos ver que muchas siguen una distribución normal, en la que la mayoría de los casos están muy cerca de la media, y según nos alejamos de la media se va reduciendo la cantidad de casos por cada valor.

Aunque la otra mitad de las variables, como `area_mean` o `concavity_mean`, siguen una distribución asimétrica positiva, es decir, que la mayoría de los casos se encuentran a la izquierda de la media, y según avanzamos a la derecha hay muchos menos casos. Podemos ver que no hay ninguna variable que siga una distribución asimétrica negativa (la mayoría de los casos se centran a la derecha de la media), siguiendo todas las variables una ***distribución normal*** o ***asimétrica positiva***. 

Destacar que ninguna variable sigue una ***distribución uniforme*** por lo que no podemos intuir que alguna variable en concreto sea inútil para la clasificación, por lo que de momento no podemos pensar en eliminar ninguna variable en el preprocesamiento, pues a priori, por su distribución vemos que todas pueden aportar algo de información.

In [ ]:
utils.plot_histogram(data_train)

A partir de este histograma se pueden sacar muchas conclusiones individuales sobre cada variable. Se pueden ver los tipos de distribuciones que sigue cada variable, aunque esto ya lo hemos visto de forma más sencilla en el gráfico anterior.

Con este histograma que es de mayor tamaño lo usaremos para detectar ***outlier***.

Para la variable `area_se` tenemos que casi todos sus valores están entre el 0 y el 180, teniendo algún caso suelto por el 200, y dos casos con un valor de 500. Como podemos ver estos casos están totalmente alejados del resto, por lo que son datos anómalos.

Para la variable `fractal_dimension_worst` podemos observar lo mismo. Sigue una distribución normal, donde sus valores van entre el 0.00 y el 0.15, alcanzando la cima de la normal en el 0.08, y tiene un valor superior al 0.2, siendo éste un dato anómalo, estando muy alejado del resto de valores que se encuentran por debajo del 0.15.

Aunque es una forma correcta de obtener información de cada variable, puede llegar a ser costoso ya que al haber tantas variables predictoras hay que mirar muchos histogramas. Otra forma de obtener este tipo de información es observando valores estadísticos de cada variable, como la media, desviación típica, el máximo...

In [ ]:
data_train.describe(include="number")

Podemos ver que a partir de los 8 valores que se nos proporciona para cada variable numérica podemos obtener gran parte de información. Aunque al tener 30 variables no nos muestran todas las columnas de la tabla (cada columna representa a una variable), por lo que para que sea más visual vamos a dividir esta tabla en 3 (teniendo 10 columnas por tabla).

In [ ]:
data_train.iloc[:,0:10].describe(include="number")

Ahora es mucho más fácil obtener información porque de un vistazo estamos viendo las características individuales de 10 variables predictoras.

Por ejemplo para `area_mean` podemos ver que tiene una desviación típica muy elevada (360), por lo que podemos deducir que sus valores están alejados de la media. Si esto lo comprobamos con el histograma correspondiente vemos que hay muchos valores muy superiores a la media. Y si además nos fijamos en que el valor máximo es 2501, y lo comparamos con el tercer cuartil, que su valor es 802, nos damos cuenta de que hemos detectado un outlier (y si lo comprobamos con el histograma podemos detectar que hay más de un outlier).

Con la variable `texture_mean` podemos detectar de la misma forma la presencia de un outlier, ya que el máximo valor es 39 y por la desviación típica (que tiene un valor muy bajo) podemos ver que los valores están cerca de la media, y la media vale 19, por lo que deducimos que 39 es muy distinto del resto de valores.

En resumen, mirando los estadísticos y los histogramas de estas 10 variables podemos ver que las variables que presentan algún outlier son:
* `texture_mean` tiene uno.
* `area_mean` tiene dos.
* `smoothness_mean` tiene uno.

In [ ]:
data_train.iloc[:,10:20].describe(include="number")

De la misma forma podemos detectar outliers, por ejemplo para `compactness_se` el caso máximo vale 0.1354 y la media y el tercer cuartil valen 0.0257 y 0.0324, por lo que tenemos un outlier. 

En el atributo `concavity_se` hay un claro outlier ya que la diferencia entre el tercer cuartil y el máximo es mucho mayor que en el otro caso. Teniendo un máximo de 0.396 y un tercer cuartil de 0.04255.

En resumen, mirando los estadísticos y los histogramas de estas 10 variables podemos ver las variables que presentan algún outlier, que en este caso son muchas más que antes:
* `radius_se` tiene dos.
* `texture_se` tiene uno.
* `perimeter_se` tiene dos.
* `area_se` tiene dos.
* `smoothness_se` tiene uno.
* `compactness_se` tiene uno.
* `concavity_se` tiene dos.
* `concave points_se` tiene uno.
* `simmetry_se` tiene uno.
* `fractal_dimension_se` tiene cuatro.

In [ ]:
data_train.iloc[:,20:30].describe(include="number")

Hemos comprado que efectivamente a partir de estos valores podemos observar lo mismo que con los histogramas anteriores. Por ejemplo, antes observamos que la variable `fractal_dimension_worst` tenía un outlier. Pues ahora con estos valores al observar que la media es 0.083971 y el tercer cuartil es 0.91865 (que son valores muy parecidos), y el máximo es 0.2075 que es muy superior al tercer cuartil, podemos ver que este valor se trata de un outlier (lo mismo que con el histograma).

En resumen, mirando los estadísticos y los histogramas de estas 10 variables podemos ver que las variables que presentan algún outlier son:
* `area_worst` tiene uno.
* `concavity_worst` tiene dos.
* `simmetry_worst` tiene uno.
* `fractal_dimension_worst` tiene dos.

### Análisis multivariado

Después de realizar el análisis univariado donde hemos detectado varios outlier, y hemos comprobado que ninguna variable sigue una distribución uniforme, vamos a realizar un análisis multivariado que nos puede ayudar a ver las relaciones que hay entre varias variables.

En concreto vamos a mostrar unos gráficos que muestran la relación entre pares de variables. Tenemos el problema de que al tener 30 variables predictoras nos van a salir cientos de gráficas, siendo imposible para nosotros obtener información de ahí.

La alternativa que vamos a realizar es dividir a las variables predictoras en grupos, e intentar buscar alguna relación entre las variables de dentro de un mismo grupo. En este caso podemos extraer 3 grupos de variables, ya que se pueden dividir en variables que miden `mean`, `standard_error` and `worst`, igual que hicimos en el análisis univariado.

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.iloc[:,0:10], y_train), target=target)

En este gráfico algo que se puede ver es que hay variables como `symmetry_mean`, `fractal_dimension_mean` o `smoothness_mean` que parece que son independientes de la variable clase, es decir, el valor de esas variables por lo que se ve en el gráfico no influyen en el valor de la clase. Este es un dato importante porque estas variables que parece que no aportan información se pueden eliminar de los datos y no tenerlas en cuenta para en el entrenamiento.

Esto se haría en el preprocesamiento, aunque en este caso como estamos trabajando con árboles de decisión que son muy buenos clasificadores, esto no influiría mucho en el rendimiento obtenido por el clasificador porque nunca seleccionaría estas variables para clasificar.

Aun así las vamos a eliminar para reducir el coste computacional, ya que tenemos demasiadas variables en este problema en concreto.

In [ ]:
# Almacenamos los nombre de las variables mean que vamos a eliminar
column_rm_mean = ["smoothness_mean", "symmetry_mean", "fractal_dimension_mean"]

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.iloc[:,10:20], y_train), target=target)

En esta ocasión podemos ver otras variables que no influyen en la variable clase, estas son: `texture_se`, `smoothness_se`, `symmetry_se` y `fractal_dimension_se`.

In [ ]:
# Almacenamos los nombre de las variables se que vamos a eliminar
column_rm_se = ["texture_se", "smoothness_se", "symmetry_se", "fractal_dimension_se"]

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.iloc[:,20:30], y_train), target=target)

Vemos que las variables `symetry_worst` y `fractal_dimension_worst` no influyen en la variable clase.

In [ ]:
# Almacenamos los nombre de las variables worst que vamos a eliminar
column_rm_worst = ["symmetry_worst", "fractal_dimension_worst"]

A lo largo de este análisis multivariado podemos ver también que algunas variables podrían funcionar mejor si se discretizaran en dos intervalos, por ejemplo para `perimeter_worst` podemos ver que si dividimos por el valor 100 (más o menos) la mayoría de los valores menores pertenecen al valor de la clase `B` y los superiores a la `M`.

En otras variables como las tres áreas (`area_mean`, `area_se` y `area_worst`) ocurre lo mismo se puede dividir en dos intervalos.

Así que otra conclusión que hemos sacado de este análisis multivariado es que en el preprocesamiento se podría realizar un discretizado de las variables en dos intervalos. Se ve claro que se puede aprender un mejor modelo si discretizamos las variables, pero no vemos tan claro el tipo de discretizado (si sería mejor discretizar por anchura o por frecuencia).

Para ver que tipo de discretizado podría funcionar mejor podríamos realizar una validación de modelos, pero como no es el objetivo de esta práctica, vamos a elegir discretizar por igual anchura, que parece que puede funcionar bien.

Ya que si elegimos discretizar por igual frecuencia, al tener una muestra desbalanceada, en el mejor de los casos tendríamos un intervalo con todos sus casos con el valor de la clase `B` y en el otro intervalo habría casos con valor `B` y `M`, dado que hay muchas mas instancias cuyo valor de la clase es `B`, que `M`.

## 3.3. Preprocesamiento de datos

Después de terminar el análisis exploratorio de datos, vamos a intentar preparar los datos a partir de las conclusiones que hemos obtenido para obtener modelo mejor entrenado.

Primero observamos que no tenemos valores perdidos ya que eliminamos la variable corrupta que se había generado por culpa del fichero, y el resto de valores predictoras no tenían ningún valor perdido.

Vamos a utilizar el concepto de pipeline para hacer las transformaciones que realicemos en el preprocesamiento. Como tendríamos que aplicar las transformaciones en los datos dos veces (una para los datos de entrenamiento, y otra para los de prueba), crearemos un pipeline indicando las transformaciones que queremos hacerle a los datos, y este pipeline se encargará de hacerlas.

### Eliminación variables

Como vimos en el análisis multivariado hay algunas variables que parece que son independientes de la variable clase. Vamos para ello a eliminarlas de los datos, porque aunque los árboles sean muy buenos clasificadores y no se obtengan demasiadas mejores, si que van a reducir el coste computacional ya que 30 variables predictoras como tenemos son muchas.

Primero vamos a visualizar las variables que dijimos que íbamos a quitar.

In [ ]:
column_rm = column_rm_mean + column_rm_se + column_rm_worst
column_rm

Ahora utilizaremos una función que se encargará de eliminar estas variables predictores, para aplicar esta función en el pipeline.

Para realizar esto utilizaremos `FunctionTransformer` de `Scikit-learn` que nos permite pasarle como parámetro una función, y aplicar esta función en el pipeline a los datos. También le indicaremos que la función recibirá como parámetro a una lista con los nombres de las variables a eliminar.

In [ ]:
drop_col = FunctionTransformer(utils.drop_column, kw_args={'columns':column_rm})

### Eliminación de outlier

En el análisis univariado detectamos la presencia de algunos outlier, que como no detectamos muchos, el tratamiento de los outlier que vamos a hacer es eliminarlos. Para ello eliminaremos la instancia entera en la que haya algún outlier.

Para ello utilizaremos `FunctionSampler` de `imblearn` al que le pasaremos como parámetro una función que hemos definido llamada `outlier_rejection`. Indicar que `imblearn` es compatible con `Scikit` por lo que se puede crear un pipeline con este módulo y los de `Scikit`.

Esta función la hemos incluido en el archivo utils y se encarga de eliminar los outlier que detecte en los datos que le pasemos como parámetro (recibirá la X y la y). Para que el resultado pueda ser reproducible, le pasamos como parámetro una semilla.

In [ ]:
outlier_rm = FunctionSampler(func=utils.outlier_rejection, kw_args={'seed':seed})

### Discretización

En el análisis multivariado vimos que el clasificador podría funcionar mejor si discretizamos las variables en dos intervalos. Vimos que el tipo de discretizado que podiamos aplicarle sería igual anchura, ya que al tener una muestra desbalanceado que tiene dos categorías, funcionaría mal el discretizado por igual frecuencia.

Utilizando el módulo de `Scikit` `KBinsDiscretizer` definiremos la transformación a aplicar en el pipeline, indicandole mediante hiperparámetros que discretice en dos intervalos por igual frecuencia.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="uniform")

## 3.4. Algoritmos de clasificación

Igual que con el otro dataset de `diabetes` de la parte anterior, en esta parte de la práctica vamos a trabajar exclusivamente con los algoritmos de clasificación ***Zero-R***, que es el clasificador más básico, prediciendo todos los valores igual (como el valor más abundante de la clase). Este clasificador lo utilizaremos para compararlo con el otro clasificador que vamos a usar, que es ***algoritmo CART***, y observar las mejoras en los resultados que obtendremos con los clasificadores generados.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Una vez que hemos definido los clasificadores que vamos a usar, podemos definir el pipeline que se encargará de aplicar las tres transformaciones que hemos definido previamente sobre el conjunto de datos.

Para ello usamos la función `make_pipeline` y le pasamos como parámetros los transformadores.

Vamos a entrenar dos modelos, uno sin discretizar y otro con discretizado. Para ello crearemos dos pipeline, uno que aplica todo el preprocesado que hemos indicado antes, pero sin aplicar el discretizado, y otro que si que aplica el discretizado.

In [ ]:
prep_tree_model = make_pipeline(drop_col, outlier_rm, tree_model)

In [ ]:
prep_tree_model_discretizer = make_pipeline(drop_col, outlier_rm, discretizer, tree_model)

## 3.5. Evaluación de modelos

Ahora que ya hemos definido los algoritmos y el pipeline podemos entrenar los modelos y evaluarlos. 

Vamos a empezar evaluando el modelo obtenido con el Zero-R que es el más básico.

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Podemos ver que ha obtenido una precisión del 62,573% clasificando todos los casos como ***B***. Ha obtenido un 62% de precisión porque como se vio en el análisis exploratorio la muestra estaba desbalanceada, esto quiere decir que había más cantidad de casos con un valor de `B` que casos con un valor de `M`.

Además, el problema de este modelo es que siempre va a predecir como `B`, fallando siempre que el resultado correcto sea `M`. Y probablemente es más importante predecir como `M` y que la predicción correcta fuese `B` que fallar la predicción al reves, no teniendo el mismo coste al fallar la predicción. Por esto un modelo de este tipo en algún problema médico no nos interesa, ya que lo que de verdad queremos es descubrir cuando el cáncer es maligno (siendo más importante descubrir cuando es maligno que cuando es benigno).

Ahora vamos a evaluar al árbol de clasificación pero sin discretizar.

In [ ]:
utils.evaluate(prep_tree_model,
               X_train, X_test,
               y_train, y_test)

Podemos ver que se ha obtenido una buena precisión, siendo esta del 87 %, fallando solo 18 casos de los 121 que ha predicho como benignos y eran malignos. Y ha fallado 4 casos de los 50 que ha predicho como malingos y eran benignos. 

Podemos ver que son buenos resultados, aunque vamos a ir más allá y dado que tenemos problema médico (se está predicieno sobre el cáncer), vamos a realizar una evaluación sensible al coste, aplicando un coste distinto según el fallo.

Dado que es un fallo mucho más grave tener a una persona que posee un cáncer maligno, y decirle que es benigno, a este fallo en la predicción le asignaremos un coste de 100. Mientras que si tenemos a una persona con un cáncer benigno y le decimos que es maligno le daremos un coste de 10. Así le estamos dando 10 veces más importancia al primer tipo de fallo.

Tendríamos una matriz de coste:

|      |   |    Coste      |   |   |
|------|---|----------|---|---|
|      | **B** |     0    | 10 |   |
| **Real**| **M** |    100   | 0 |   |
|      |   |     **B**    | **M** |   |
|      |   | **Predicho**   |   |

Si aplicaramos esta matriz de coste a los resultados predichos por el modelo tendríamos.

$$Coste = 0*103 + 10*4 + 100*18 + 0*46 = 1840$$

Hemos tenido un coste un poco elevado relacionado con la precisión obtenida del modelo porque se han obtenido más casos predichos como benignos, que en realidad eran malignos que al reves. En este tipo de modelos nos interesaría reducir la cantidad de estos casos, aunque nos cueste aumentar (un poco) los casos predichos como malignos, pero que su valor real sea benigno.

Por último vamos a evaluar al modelo discretizado.

In [ ]:
utils.evaluate(prep_tree_model_discretizer,
               X_train, X_test,
               y_train, y_test)

Como era de esperar este modelo ha obtenido una mayor precisión, casi del 93%, afirmando que igual que vimos en el análisis multivariado, si discretizábamos a las variables en dos intervalos se podían obtener mejores resultados.

Ahora utilizando la misma matriz de coste que se definió para calcular el coste del modelo anterior, vamos a ver cuanto se ha reducido el coste en este modelo.
$$Coste = 0*105 + 10*2 + 100*10 + 0*54 = 1020$$

También hemos obtenido una gran mejora en el coste (antes teníamos 1840, y se ha reducido casi a la mitad) ya que se han reducido los casos predichos como benignos pero que en realidad eran malignos, que para este problema en concreto es muy importante reducir este tipo de casos.